# Unfitted Finite Element Method for the Stokes Problem

In this notebook, we solve the Stokes equations on an **unfitted mesh** using the **ghost penalty stabilization** technique. This approach allows us to handle complex geometries without conforming the mesh to the domain boundary.

The Stokes equations in strong form are given by:

$$
\begin{aligned}
- \Delta \mathbf{u} + \nabla p &= \mathbf{f} \quad \text{in } \Omega, \\
\text{div}(\mathbf{u}) &= 0 \quad \text{in } \Omega, \\
\mathbf{u} &= \mathbf{g} \quad \text{on } \partial\Omega,
\end{aligned}
$$

where: 
- $ \mathbf{u} $ is the velocity field,
- $ p $ is the pressure,
- $ \mathbf{f} $ is a given forcing term,
- and $ \mathbf{g} $ is the prescribed Dirichlet boundary condition.

In [213]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
import ngsolve
from xfem import *

In [214]:
square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=0.05)
mesh = Mesh(ngmesh)
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## Fictitious Domain and Level Set Representation

We work with a **fixed background mesh** $\widehat{\Omega} = [-1.25, 1.25]^2$, which is independent of the actual geometry.

The **physical domain** $\Omega$ is defined implicitly via a **level set function** $\phi(x, y)$, and corresponds either to the **interior** or **exterior** of a closed boundary $\Gamma = \{ \phi = 0 \}$.

- If $\phi(x, y) < 0$, the point $(x, y)$ lies **inside** $\Omega$
- If $\phi(x, y) > 0$, the point lies **outside**
- $\Gamma$ is the zero level set and represents the **interface**

This approach allows us to define complex geometries without modifying the mesh. We solve the Stokes problem on $\Omega$ using an unfitted finite element method, where the background mesh is cut by $\Gamma$, and stability is ensured using **ghost penalty stabilization**.


In [215]:
r = sqrt(x**2 + y**2)
levelset = r-1
lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
DrawDC(levelset, CF(1), CF(0), mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [216]:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
neg = ci.GetElementsOfType(NEG)
hasif = ci.GetElementsOfType(IF)
haspos = ci.GetElementsOfType(HASPOS)
ba_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=any)
Draw(BitArrayCF(hasneg),mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## Fictitious Domain Construction

The **fictitious domain** $\Omega^*$ is defined as the union of all background mesh elements that intersect the physical domain $\Omega$. That is, we consider the minimal subset of elements $\mathcal{T}^* \subset \widehat{\mathcal{T}}$ such that:

$$
\Omega \subset \bigcup_{T \in \mathcal{T}^*} T =: \Omega^*,
$$

where:
- $\widehat{\mathcal{T}}$ denotes the background mesh over $\widehat{\Omega}$,
- $\mathcal{T}^*$ is the collection of all elements that are **cut by** or **lie inside** the level set domain $\Omega$.

This extended domain $\Omega^*$ is used for the unfitted finite element formulation. Integration and stabilization terms are evaluated over $\Omega^*$ instead of $\Omega$, which avoids the need for boundary-fitted meshes.


In [217]:
interface_facet_set = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasneg)
dx = dCut(lsetp1, NEG, definedonelements=hasneg)
ds = dCut(lsetp1, IF, definedonelements=hasif)
ds_inner_facets = dCut(lsetp1, NEG, definedonelements=interface_facet_set, skeleton=True)
dw_interface = dFacetPatch(definedonelements=interface_facet_set)

## Interior Facets Near the Interface

We introduce the notation $\mathcal{F}_\Gamma^*$ for the set of all **interior facets** that belong to elements **intersected by the interface** $\Gamma$. This set plays a key role in the definition of the **ghost penalty stabilization**.

Formally, we define:

$$
\mathcal{F}_\Gamma^* = \left\{ F \in \partial_i \mathcal{T}^* \; : \; T_F^+ \cap \Gamma \neq \emptyset \; \text{or} \; T_F^- \cap \Gamma \neq \emptyset \right\},
$$

where:
- $\mathcal{T}^*$ is the set of active (cut or inside) elements covering the physical domain $\Omega$,
- $\partial_i \mathcal{T}^*$ denotes the set of **interior facets** (i.e., shared by two elements),
- $T_F^+$ and $T_F^-$ are the two elements sharing facet $F$.

This set $\mathcal{F}_\Gamma^*$ identifies all facets in the vicinity of the interface $\Gamma$ and is used as the integration domain for ghost penalty terms.


## Weak Formulation using P1–P1 Elements

We now reformulate the Stokes problem in its **weak (variational) form**. To this end, we use the classical **$P_1$–$P_1$** finite element discretization, i.e., continuous linear elements for both velocity and pressure.

We seek a pair of functions $(u_h, p_h) \in V_h \times Q_h$, such that

$$
A_h(u_h, p_h; v_h, q_h) = L_h(v_h, q_h) \quad \text{for all } (v_h, q_h) \in V_h \times Q_h,
$$

with $V_h \times Q_h = P_1 \times P_1$, and where the bilinear and linear forms are defined by:

$$
A_h(u_h, p_h; v_h, q_h) = a_h(u_h, v_h) + b_h(u_h, q_h) + b_h(v_h, p_h) - c_h(u_h, p_h; q_h),
$$

$$
L_h(v_h, q_h) = (f, v_h) - \Phi_h(q_h).
$$

### Definitions of the bilinear and linear forms:

- $a_h(u_h, v_h) = (\nabla u_h, \nabla v_h)_\Omega - (\partial_n u_h, v_h)_\Gamma - (\partial_n v_h, u_h)_\Gamma + \gamma h^{-1} (u_h, v_h)_\Gamma$
- $b_h(v_h, p_h) = -(\nabla \cdot v_h, p_h)_\Omega + (n \cdot v_h, p_h)_\Gamma$
- $c_h(u_h, p_h; q_h) = \beta_1 \sum_{T \in \mathcal{T}_h} h_T^2 (-\Delta u_h + \nabla p_h, \nabla q_h)_T$
- $\Phi_h(q_h) = \beta_1 \sum_{T \in \mathcal{T}_h} h_T^2 (f, \nabla q_h)_T$

Here, $f$ is the given forcing term, and $\gamma$ is the Nitsche penalty parameter. The stabilization terms $c_h$ and $\Phi_h$ are essential to ensure stability of the equal-order discretization and are commonly used in pressure-Poisson stabilized Galerkin methods.


In [218]:
V = VectorH1(mesh, order=1,dgjumps=True)
V = Compress(V, GetDofsOfElements(V,hasneg))
Q = H1(mesh, order=1)
Q = Compress(Q, GetDofsOfElements(Q,hasneg))
Z = NumberSpace(mesh)
X = V*Q*Z
(u,p,z),(v,q,z1) = X.TnT()
gfu = GridFunction(X)
h = specialcf.mesh_size
n = Normalize(grad(lsetp1))
gamma_stab = 100
beta1 = 1

def jump(f):
        return f - f.Other()
def jumpn(f):
        n = Normalize(grad(lsetp1))
        return Grad(f)*n - Grad(f).Other()*n

stokes = BilinearForm(X)
stokes += InnerProduct(Grad(u), Grad(v))*dx - div(u)*q*dx - div(v)*p*dx + (q*n * u + p*n * v) * ds
stokes += -(Grad(u)*n * v + Grad(v)*n * u) * ds + gamma_stab / h * u * v * ds #Nitsche term
stokes += p*z1 *dx + q *z*dx
stokes += -beta1 * h**2 * Grad(p) * Grad(q) * dx #Stabilization term

## Analytical Test Case: Do We Need Ghost Penalty?

Now let us construct an analytical example and check whether our unfitted Stokes discretization indeed requires a **ghost penalty stabilization** for stability and convergence.

We define the exact velocity and pressure solutions as:

$$
u_{\text{exact}}(x, y) = 
\begin{pmatrix}
\sin(\pi x) \cos(\pi y) \\
-\cos(\pi x) \sin(\pi y)
\end{pmatrix}, \quad
p_{\text{exact}}(x, y) = \sin(\pi x) \sin(\pi y).
$$

We then compute the corresponding right-hand side $f$ using the Stokes equation $-\Delta u + \nabla p = f$:

- The Laplacian of $u_{\text{exact}}$ (component-wise):

$$
\Delta u_x = -2\pi^2 \sin(\pi x) \cos(\pi y), \quad
\Delta u_y = 2\pi^2 \cos(\pi x) \sin(\pi y),
$$

- The gradient of $p_{\text{exact}}$:

$$
\frac{\partial p}{\partial x} = \pi \cos(\pi x) \sin(\pi y), \quad
\frac{\partial p}{\partial y} = \pi \sin(\pi x) \cos(\pi y),
$$

- The full forcing term becomes:

$$
f(x, y) = 
\begin{pmatrix}
- \Delta u_x + \frac{\partial p}{\partial x} \\
- \Delta u_y + \frac{\partial p}{\partial y}
\end{pmatrix}.
$$

We will use this exact solution to evaluate the accuracy of our unfitted method with and without ghost penalty terms, and observe whether instability or locking occurs in the absence of proper stabilization.


In [219]:
uexact = CoefficientFunction((
    sin(pi*x) * cos(pi*y),   # u(x,y)
    -cos(pi*x) * sin(pi*y)   # v(x,y)
))
pexact = sin(pi*x) * sin(pi*y)
# Laplace von uexact (Komponentenweise)
lapu_x = -pi**2 * sin(pi*x) * cos(pi*y) * 2
lapu_y = -pi**2 * (-cos(pi*x) * sin(pi*y)) * 2  # Minuszeichen kommt von v

# Gradient von p
dp_dx = pi * cos(pi*x) * sin(pi*y)
dp_dy = pi * sin(pi*x) * cos(pi*y)

# Gesamte rechte Seite f = -Δu + ∇p
f = CoefficientFunction((
    -lapu_x + dp_dx,
    -lapu_y + dp_dy
))
u1 = -4*y * (1 - x**2 - y**2)
u2 = 4*x*(1 - x**2 - y**2)
uexact = CoefficientFunction((u1, u2))
pexact = sin(x)*cos(y)
# Laplace-Anteil
lapu1 = 32*y
lapu2 = -32*x
# Gradient von p
dpdx = cos(x) * cos(y)
dpdy = -sin(x) * sin(y)
f = CoefficientFunction((
    -lapu1 + dpdx,
    -lapu2 + dpdy
))

In [220]:
rhs = LinearForm(X)
rhs += InnerProduct(f, v) * dx 
rhs += -h**2 * f * Grad(q) * dx # phi
rhs += -(Grad(v)*n * uexact) * ds + gamma_stab / h * uexact * v * ds+q*n*uexact *ds

In [221]:
a = stokes
a.Assemble()
b = rhs
b.Assemble()

gfu.vec.data = a.mat.Inverse(freedofs=X.FreeDofs()) * b.vec
DrawDC(levelset, gfu.components[0], CF((0,0)),mesh, "velocity")
print("L2 error velocity:", sqrt(Integrate((gfu.components[0] - uexact)**2*dx, mesh)))
print("L2 error pressure:", sqrt(Integrate((gfu.components[1] - pexact)**2*dx, mesh)))

webgui discontinuous vis only for scalar functions a.t.m., switching to IfPos variant


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

L2 error velocity: 0.012556359247557448
L2 error pressure: 0.09037348911591356


## Adding Ghost Penalty Stabilization

Now let us add the **ghost penalty terms** to stabilize the unfitted Stokes formulation. These terms penalize jumps of normal derivatives across interior facets that are intersected by the geometry, ensuring stability even when elements are only partially in the physical domain.

We define the ghost penalty contribution as

$$
J_h(u_h, p_h; v_h, q_h) = i_h(u_h, v_h) - j_h(p_h, q_h),
$$

where:

- The **velocity ghost penalty** is given by

$$
i_h(u_h, v_h) = \beta_2 \sum_{F \in \mathcal{F}_\Gamma^*} h_F \left( [\partial_n u_h], [\partial_n v_h] \right)_F,
$$

- The **pressure ghost penalty** is given by

$$
j_h(p_h, q_h) = \beta_3 \sum_{F \in \mathcal{F}_\Gamma^*} h_F^3 \left( [\partial_n p_h], [\partial_n q_h] \right)_F.
$$

Here:

- $\mathcal{F}_\Gamma^*$ is the set of interior facets belonging to elements cut by the boundary $\Gamma$ (as defined above),
- $[\cdot]$ denotes the jump of a quantity across a facet,
- $\partial_n$ denotes the normal derivative,
- $h_F$ is a measure of the facet size,
- and $\beta_2$, $\beta_3$ are positive stabilization parameters.

These terms are added to the bilinear form to improve robustness and ensure well-posedness in the unfitted setting.


In [222]:
beta2 = 1
beta0 = 1
stokes += beta2* h* InnerProduct(jumpn(u), jumpn(v)) * ds_inner_facets #velocity ghost penalty
#stokes += beta2 * h**-2* InnerProduct(jump(u), jump(v)) * dw_interface #velocity ghost penalty
stokes += -beta0 * h**3*InnerProduct(jumpn(p), jumpn(q)) * ds_inner_facets #pressure ghost penalty
#stokes += -beta0 * InnerProduct(jump(p), jump(q)) * dw_interface
stokes.Assemble()
rhs.Assemble()
gfu1 = GridFunction(X)
gfu1.vec.data = stokes.mat.Inverse(freedofs=X.FreeDofs()) * rhs.vec
DrawDC(levelset, gfu1.components[0], CF((0,0)),mesh, "velocity with ghost penalty")
print("L2 error velocity with ghost penalty:", sqrt(Integrate((gfu1.components[0] - uexact)**2*dx, mesh)))
print("L2 error pressure with ghost penalty:", sqrt(Integrate((gfu1.components[1] - pexact)**2*dx, mesh)))
DrawDC(levelset,uexact, CF((0,0)), mesh, "exact velocity")

webgui discontinuous vis only for scalar functions a.t.m., switching to IfPos variant


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

L2 error velocity with ghost penalty: 0.012146632455498968
L2 error pressure with ghost penalty: 0.09711363595975313
webgui discontinuous vis only for scalar functions a.t.m., switching to IfPos variant


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…